<a href="https://colab.research.google.com/github/amuradbegovic/rang-lista-svjetskih-univerziteta-vis/blob/main/VIS25-Z2-TIM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zadaća 2 - Statistička analiza svjetskih univerziteta

Tim 1: **Adnan Muradbegović, Aid Mustafić, Tarik Redžić, Zlatan Ljutika**

---
U ovom projektu, baviti ćemo se statističkom analizom _[Times Higher Education](https://www.timeshighereducation.com)_ rang liste svjetskih univerziteta za 2024. godinu.


Izvor podataka: <br>
**[Dosad, D. (2024), Times World University Rankings 2024](https://www.kaggle.com/datasets/ddosad/timesworlduniversityrankings2024)**



In [ ]:
# import potrebnih python biblioteka
import pandas as pd                   ## Za manipulisanje golih podataka, (CSV, DataFrame,...)
import matplotlib.pyplot as plt       ## Za vizualizaciju podataka
import seaborn as sns                 ## Za vizualizaciju podat
import numpy as np                    ## Za izvršavanje matematičkih operacija nad podacima
from numpy import cov
import statistics

df = pd.read_csv("TIMES_WorldUniversityRankings_2024.csv")


### 1. Deskriptivna statistika i vizualizacija distribucija varijabli

Član tima: **Tarik Redžić**

---

### 3. Chi-

Član tima: **Tarik Redžić**

---

### <p>2. <a href="https://en.wikipedia.org/wiki/Probability_distribution_fitting" style="color:inherit">Prilagođavanje distribucija vjerovatnoće podataka</a> <br> _(eng. Probability distribution fitting)_</p>

Član tima: **Aid Mustafić**

---

### 1. Deskriptivna statistika i vizualizacija distribucija varijabli

Član tima: **Tarik Redžić**

---

### 1. Deskriptivna statistika i vizualizacija distribucija varijabli

Član tima: **Tarik Redžić**

---